In [41]:
from datasets import load_dataset
from datasets import get_dataset_config_names
from collections import defaultdict
from datasets import DatasetDict
import pandas as pd

# Meata data

In [33]:
datasets = 'xtreme'
sub_datasets = 'PAN-X.'
NA_name = ['de','fr','it','en']
fracs=[0.629,0.229,0.084,0.059]
num_seed = 0

In [18]:
datasets_list=get_dataset_config_names(datasets)
panx=[s for s in datasets_list if s.startswith(sub_datasets)]

In [21]:
ds=load_dataset(datasets, name=f"PAN-X.ko")

# 필요한 데이터 학습 데이터 생성

In [38]:
panx_ds=defaultdict(DatasetDict)

for na, frac in zip(NA_name, fracs):
    ds=load_dataset(datasets, name=f"PAN-X.{na}")
    for split in ds:
        panx_ds[na][split]=(
            ds[split].shuffle(seed=num_seed)
                 .select(range(int(frac * ds[split].num_rows)))
        )

# train data len

In [52]:
pd.DataFrame({na:[panx_ds[na]['train'].num_rows] for na in NA_name},
            ['train data_num'])

,de,fr,it,en
train data_num,12580,4580,1680,1180


# data example

In [57]:
elements=panx_ds['de']['train'][0]
for k,v in elements.items():
    print({k:v})

{'tokens': ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']}
{'ner_tags': [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]}
{'langs': ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']}


# data info

In [60]:
for k,v  in panx_ds['de']['train'].features.items():
    print({k:v})

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}
{'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}
{'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


# add data features(ner int 2 str)

In [90]:
tag=panx_ds['de']['train'].features['ner_tags'].feature

def ner_create_name(batch):
    return {'ner_str_name':[tag.int2str(index) for index in batch['ner_tags']]}
for ns in NA_name:
    panx_ds[ns] = panx_ds[ns].map(ner_create_name)

In [91]:
for k,v in panx_ds['de']['train'][0].items():
    print({k:v})

{'tokens': ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']}
{'ner_tags': [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]}
{'langs': ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']}
{'ner_str_name': ['O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'B-LOC', 'B-LOC', 'I-LOC', 'O']}


In [107]:
pd.DataFrame([panx_ds['de']['train'][0]['tokens'], panx_ds['de']['train'][0]['ner_str_name']],
            index = ['tokens','ner_str_name'])

,0,1,2,3,4,5,6,7,8,9,10,11
tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
ner_str_name,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [110]:
from collections import Counter

split2freqs = defaultdict(Counter)
for split,dataset in panx_ds['de'].items():
    for row in dataset['ner_str_name']:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[-1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs)

,train,validation,test
LOC,6186,3172,3180
ORG,5366,2683,2573
PER,5810,2893,3071


In [ ]:
from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset['ner_tags_str']:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient='index')                
    